In [44]:
# Opens a client to Google BigQuery to pull the data
from google.cloud import bigquery
from google.oauth2 import service_account
from datetime import date, timedelta
import pandas as pd
from collections import defaultdict
import numpy as np
import networkx as nx
import pandas as pd
import math
import igraph as ig

key_path = '/Users/lilyfrancus/Downloads/service_key_google_cloud.json'

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
"""
Find the starting date on the Ethereum blockchain where txn data exists (the first day).
"""
def find_starting_date():
    query_string = """
    
    SELECT block_timestamp from `bigquery-public-data.crypto_ethereum.transactions`
    where block_number = 
        (select min(block_number) from `bigquery-public-data.crypto_ethereum.transactions`)
    """
    return client.query(query_string).result().to_dataframe()

"""
Given two dates, query the bigquery-public-data for the relevant transaction data
"""
def find_all_txns_on_date(start, end):
    start_timestr = start.strftime('%Y-%m-%d')
    end_timestr = end.strftime('%Y-%m-%d')
    query_string = f"""

    SELECT *
    FROM `bigquery-public-data.crypto_ethereum.transactions`
    WHERE block_timestamp BETWEEN '{start_timestr}' AND '{end_timestr}'
    """
    return client.query(query_string).result().to_dataframe()

def get_start_plus_n_day_txns(n: int):    
    start_of_blockchain = find_starting_date()
    day = start_of_blockchain.iloc[0]['block_timestamp'].date() + timedelta(days=n)
    next_day = day + timedelta(days=1)
    zf = find_all_txns_on_date(day, next_day)
    day_as_timestamp = pd.to_datetime(day).timestamp()
    return zf, day_as_timestamp

MINUTES_PER_FRAME = 8
TIMEFRAME_IN_SECONDS = MINUTES_PER_FRAME*60

def specialized_round(x, base=0.02):
    return base * round(x/base)


def fit_data(df: pd.DataFrame, day_as_timestamp):
    df = df.fillna(0)
    df['is_contract'] = df.apply(lambda row: row['receipt_contract_address'] and not row['to_address'], axis=1)
    df['value'] = df['value'].apply(float)
    df['receipt_effective_gas_price'] = df['receipt_effective_gas_price'].apply(float)
    df['receipt_cumulative_gas_used'] = df['receipt_cumulative_gas_used'].apply(float)
    df['to_address'] = df.apply(lambda row: row['to_address'] if row['to_address'] else row['receipt_contract_address'], axis=1)
    df['true_transfer_volume'] = df['value'] + df['receipt_effective_gas_price']*df['receipt_cumulative_gas_used']
    df['grouping'] = df['block_timestamp'].apply(lambda x: math.floor((x.timestamp() - day_as_timestamp) / TIMEFRAME_IN_SECONDS))
    df['to-from'] = df['from_address'] + '-' + df['to_address']
    df['from-to'] = df['to_address']  + '-' + df['from_address']
    df = df[['grouping', 'to-from', 'from-to', 'from_address', 'to_address', 'block_number', 'block_timestamp', 'true_transfer_volume', 'is_contract', 'transaction_type', 'receipt_status']]
    max_txn_volume = float(max(df['true_transfer_volume']))
    df['inverse_exponential_vol_alpha'] = df['true_transfer_volume'].apply(lambda x: 
        specialized_round(max(
            (
            ((float(x)/ max_txn_volume) ** (1/5) * 1), 
            0
            )
        ))
    )

    pivot = df.groupby(['from_address', 'to_address']).agg({ 'true_transfer_volume': sum})
    pivot.to_csv('temp.csv')
    pivot = pd.read_csv('temp.csv')
    daily_transfers = defaultdict(float)
    for idx, row in pivot.iterrows():
        from_add = row['from_address']
        to_add = row['to_address']
        normal = f"{from_add}-{to_add}"
        inverse = f"{to_add}-{from_add}"
        if daily_transfers[inverse] > 0:
            daily_transfers[inverse] = daily_transfers[inverse] + row['true_transfer_volume']
        else:
            daily_transfers[normal] = daily_transfers[normal] + row['true_transfer_volume']
    pds = []
    all_k = []
    total_k = int((1440*60)/TIMEFRAME_IN_SECONDS)
    for k in range(total_k):
        time_slice = df[df['grouping']==int(k)]
        tos = set(time_slice['to-from']).union(set(time_slice['from-to']))
        active_nodes = set([a for b in tos for a in b.split('-')])
        opacities = defaultdict(bool)
        for index, row in time_slice.iterrows():
            if opacities[row['from-to']] > 0:
                opacities[row['from-to']] = True
            else:
                opacities[row['to-from']] = True
        all_k.append(opacities)
    for key, value in daily_transfers.items():
        source, target = key.split('-')
        if value > 0:
            basics = {"source": source, "target": target, "weight": value/10**18}
            for k in range(total_k):
                basics[f"frame_{k}"] = all_k[k][key]
            pds.append(basics)
    return pd.DataFrame(pds)

df, day_as_time = get_start_plus_n_day_txns(n=500)
txns_for_day = fit_data(df, day_as_time)
txns_for_day.to_csv(f'txns_for_day_{pd.to_datetime(day_as_time * 10 ** 9)}.csv')
g = ig.Graph.DataFrame(txns_for_day, directed=False)
d = [{"name": v.attributes()['name'], "degree": v.degree() } for v in list(g.vs)]
pd.DataFrame(d).to_csv(f'node_list_degrees_{pd.to_datetime(day_as_time * 10 ** 9)}.csv')


[{'name': '0x000001f568875f378bf6d170b790967fe429c81a', 'degree': 3},
 {'name': '0x00000217d2795f1da57e392d2a5bc87125baa38d', 'degree': 1},
 {'name': '0x000056c2e8f8e8e7c47f8f919a5d3d4190e83765', 'degree': 2},
 {'name': '0x0003562d47673f11bd15991afdd3b16989334d00', 'degree': 1},
 {'name': '0x00088b54b66dcd0b21dfd6aacef1417eeabcbbde', 'degree': 1},
 {'name': '0x000a699e8905034651be5919a8c1e2877cd59ae4', 'degree': 1},
 {'name': '0x001654ee60521ffa911258afe60148d2692866d5', 'degree': 1},
 {'name': '0x001a6ab3a96e33aaa6a176dac86c2e7780667d42', 'degree': 1},
 {'name': '0x001b9c9e8ac9c15976630e4526cc1ff966e9c34a', 'degree': 2},
 {'name': '0x0021171980ded91753879ec602ab04d52c4f4c7d', 'degree': 1},
 {'name': '0x002875b7b33003edacb8135186a038f2fe0fdd81', 'degree': 2},
 {'name': '0x002c7357f9771b69ebc89b1927ccb72e06565bc3', 'degree': 1},
 {'name': '0x0031ead73ed44052550bf6670468e42848e38be1', 'degree': 2},
 {'name': '0x00327b1460f36053295343a57c1682ab72fc59e1', 'degree': 1},
 {'name': '0x0034732